In [2]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Class representing a 2D Ising model
class Ising:
    def __init__(self, N, J, H):
        self.lattice = np.random.choice([-1,1], size=(N,N)) # initial state
        self.N = N # dimension
        self.J = np.abs(J) # interaction energy, non-negative by definition
        self.H = H # external field

In [3]:
# Function to calculate energy change when spin(i,j) is flipped, assuming periodic boundary conditions
def energy_change_of_flip(state, pos_i, pos_j):
    return 2*state.lattice[pos_i,pos_j]*(state.J*(state.lattice[pos_i,(pos_j+1)%state.N] + state.lattice[(pos_i-1)%state.N,pos_j] + state.lattice[(pos_i+1)%state.N,pos_j] + state.lattice[pos_i,(pos_j-1)%state.N])+state.H)

In [4]:
# Monte Carlo sweep using Metropolis algorithm
def monte_carlo_sweep(state, temperature):
    for i in range(len(state.lattice)**2):
        # pick a spin randomly and calculate energy change if it's flipped
        pos_i = np.random.randint(0,len(state.lattice))
        pos_j = np.random.randint(0,len(state.lattice))
        delta_E = energy_change_of_flip(state,pos_i,pos_j)
        
        # to flip or not to flip?
        if delta_E < 0 or np.exp(-delta_E/temperature) > np.random.random():
            state.lattice[pos_i,pos_j] *= -1

In [5]:
# Function to calculate magnetisation
def calculate_magnetisation(state):
    return np.sum(state.lattice)

In [6]:
# Function to calculate energy
def calculate_energy(state):
    interaction_energy = 0.0
    for pos_i in range(len(state.lattice)):
        for pos_j in range(len(state.lattice)):
            interaction_energy += -state.J*state.lattice[pos_i,pos_j]*(state.lattice[pos_i,(pos_j+1)%state.N] + state.lattice[(pos_i-1)%state.N,pos_j] + state.lattice[(pos_i+1)%state.N,pos_j] + state.lattice[pos_i,(pos_j-1)%state.N])
    interaction_energy *= 0.5 # avoid double counting
    
    magnetisation_energy = -state.H*calculate_magnetisation(state)
    
    return interaction_energy+magnetisation_energy

In [ ]:
# Dependence of mean magnetisation and energy on temperature, H=0
N_range = [2, 8, 14, 20]
J = 1.0
H = 0.0

temperature_range = np.linspace(0.5,4.0,50)  # 40 values of temperature between 0.5 and 4.0
number_of_equilibrating_sweeps = 1000
number_of_sampling_sweeps = 500

# arrays to store average energy, magnetisation, specific heat, susceptibility
average_energy = np.zeros((len(N_range),len(temperature_range)))
average_magnetisation = np.zeros((len(N_range),len(temperature_range)))
average_heat_capacity = np.zeros((len(N_range),len(temperature_range)))
average_susceptibility = np.zeros((len(N_range),len(temperature_range)))

for N_index in range(len(N_range)):
    for t_index in range(len(temperature_range)):
        state = Ising(N_range[N_index],J,H)

        # wait for the system to settle down
        for sweep in range(number_of_equilibrating_sweeps):
            monte_carlo_sweep(state,temperature_range[t_index])
            if sweep%100 == 0: # output checkpoint
                print('T = {0}, equilibrating step {1}'.format(temperature_range[t_index],sweep))

        sum_of_energy = 0.0
        sum_of_energy_squared = 0.0
        sum_of_magnetisation = 0.0
        sum_of_magnetisation_squared = 0.0

        # start sampling data
        for sweep in range(number_of_sampling_sweeps):
            monte_carlo_sweep(state,temperature_range[t_index])

            # store data
            E = calculate_energy(state)
            M = np.abs(calculate_magnetisation(state))
            sum_of_energy += E
            sum_of_energy_squared += E**2
            sum_of_magnetisation += M
            sum_of_magnetisation_squared += M**2

            if sweep%100 == 0: # output checkpoint
                print('T = {0}, sampling step {1}'.format(temperature_range[t_index],sweep))

        # average over number of sweeps
        average_energy[N_index,t_index] = (sum_of_energy/number_of_sampling_sweeps)/(state.N**2)
        average_magnetisation[N_index,t_index] = (sum_of_magnetisation/number_of_sampling_sweeps)/(state.N**2)
        average_heat_capacity[N_index,t_index] = ((sum_of_energy_squared/number_of_sampling_sweeps-(sum_of_energy/number_of_sampling_sweeps)**2)/(temperature_range[t_index]**2))/(state.N**2)
        average_susceptibility[N_index,t_index] = ((sum_of_magnetisation_squared/number_of_sampling_sweeps-(sum_of_magnetisation/number_of_sampling_sweeps)**2)/temperature_range[t_index])/(state.N**2)
    
    print('N = {0} done'.format(N_range[N_index]))

In [ ]:
# Plot of energy, magnetisation, heat capacity, susceptibility against temperature
for N_index in range(len(N_range)):
    plt.plot(temperature_range,average_energy[N_index,:],'-o',label='N = {0}'.format(N_range[N_index]))
plt.legend(loc='upper right')
plt.xlabel('Temperature')
plt.ylabel('Energy')
plt.savefig('plots/Energy_vs_Temperature_zeroH.png')
plt.figure()

for N_index in range(len(N_range)):
    plt.plot(temperature_range,average_magnetisation[N_index,:],'-o',label='N = {0}'.format(N_range[N_index]))
plt.legend(loc='upper right')
plt.xlabel('Temperature')
plt.ylabel('Magnetisation')
plt.savefig('plots/Magnetisation_vs_Temperature_zeroH.png')
plt.figure()

for N_index in range(len(N_range)):
    plt.plot(temperature_range,average_heat_capacity[N_index,:],'-o',label='N = {0}'.format(N_range[N_index]))
plt.legend(loc='upper right')
plt.xlabel('Temperature')
plt.ylabel('Heat Capacity')
plt.savefig('plots/Heat_Capacity_vs_Temperature_zeroH.png')
plt.figure()

for N_index in range(len(N_range)):
    plt.plot(temperature_range,average_susceptibility[N_index,:],'-o',label='N = {0}'.format(N_range[N_index]))
plt.legend(loc='upper right')
plt.xlabel('Temperature')
plt.ylabel('Susceptibility')
plt.savefig('plots/Susceptibility_vs_Temperature_zeroH.png')
plt.figure()

In [7]:
# Metropolis algorithm
def Metropolis_algorithm(N_range, J, H, T_range, no_of_sweeps):
    # arrays to store magnetisation and energy time series
    M = np.zeros((len(N_range),len(T_range),no_of_sweeps))
    E = np.zeros((len(N_range),len(T_range),no_of_sweeps))
    M_normalized = np.zeros((len(N_range),len(T_range),no_of_sweeps))
    E_normalized = np.zeros((len(N_range),len(T_range),no_of_sweeps))
    
    # Monte Carlo sweeps
    for N_index in range(len(N_range)):
        for T_index in range(len(T_range)):
            state = Ising(N_range[N_index],J,H)
            for sweep in range(no_of_sweeps): # run!
                monte_carlo_sweep(state,T_range[T_index])
                
                M[N_index,T_index,sweep] = calculate_magnetisation(state)
                E[N_index,T_index,sweep] = calculate_energy(state)
                M_normalized[N_index,T_index,sweep] = M[N_index,T_index,sweep]/(state.N**2)
                E_normalized[N_index,T_index,sweep] = E[N_index,T_index,sweep]/(state.N**2)
                
                if sweep%100 == 0: # output checkpoints
                    print('N = {0}, T = {1}, sweep = {2}'.format(N_range[N_index],T_range[T_index],sweep))
    
    return M, E, M_normalized, E_normalized

In [8]:
# Plot evolution of data over time
def plot_time_series(data, N_range, T_range, yname):
    for N_index in range(len(N_range)):
        for T_index in range(len(T_range)):
            plt.plot(range(len(data[N_index,T_index,:])),data[N_index,T_index,:],label='N = {0}, T = {1}'.format(N_range[N_index],T_range[T_index]))
        plt.legend(loc='best')
        plt.xlabel('Sweeps')
        plt.ylabel(yname)
        plt.savefig('plots/{0} vs Sweeps {1}.png'.format(yname,N_range[N_index]))
        plt.figure()

In [ ]:
# Set parameters
Ns = [4, 8, 12]
J = 1.0
H = 0.0
Ts = np.linspace(0.5,4.0,50)
sweeps = 2000

# Run the experiment
magnetisation, energy, magnetisation_per_site, energy_per_site = Metropolis_algorithm(Ns,J,H,Ts,sweeps)

In [10]:
# Plot time evolution
plot_time_series(magnetisation_per_site,Ns,Ts,'Magnetisation per site')

NameError: name 'magnetisation_per_site' is not defined

In [28]:
a = 3000
a%3000

0